# 1. Read Dataset(Target Stories)

In [506]:
import glob
file_names = glob.glob("/Users/ibrahim/Desktop/Preprocess/query/*.txt")
raw_documents=[]
for file in file_names:
    try:
        with open(file, "r", encoding="utf-8") as f: raw_documents.append(f.read())
    except:
        pass
print("Number of documents: ",len(raw_documents))

Number of documents:  18


In [507]:
#raw_documents

##  1.1 Read Dataset(Query Story)

In [508]:
querydoc=glob.glob("/Users/ibrahim/Desktop/Preprocess/AllStories/*.txt")
query_documents=[]
for file in querydoc:
    try:
        with open(file, "r", encoding="utf-8") as f: query_documents.append(f.read())
    except:
        pass
query = query_documents[0].lower()
#print("Number of documents: ",len(query_documents))

# Print the Query

In [509]:
query_documents

['When it Was the Ninth Night,\n\n\x0cShe said, It hath reached me, O auspicious King, that when the\nyoung woman, the sorceress, took in hand some of the tarn water\nand spake over it words not to be understood, the fishes lifted\ntheir heads and stood up on the instant like men, the spell on\nthe people of the city having been removed. What was the lake\nagain became a crowded capital; the bazars were thronged with\nfolk who bought and sold; each citizen was occupied with his own\ncalling and the four hills became islands as they were whilome.\nThen the young woman, that wicked sorceress, returned to the King\nand (still thinking he was the negro) said to him, O my love!\nstretch forth thy honoured hand that I may assist thee to rise."\n"Nearer to me," quoth the King in a faint and feigned tone. She\ncame close as to embrace him when he took up the sword lying hid\nby his side and smote her across the breast, so that the point\nshowed gleaming behind her back. Then he smote her a sec

# 1.1 Locality Sensitive Hashing (LSH) Algorithm (First Layer)
https://nbviewer.org/github/bassimeledath/quora_profile/blob/master/questions_analysis.ipynb

https://www.pinecone.io/learn/locality-sensitive-hashing/

In [510]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

In [511]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    tokens = text
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in tokens:
        if ele in punc:
            tokens = tokens.replace(ele, "")
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [512]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    #print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [513]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    result={}
    result = database.iloc[idx_array]
    
    #print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [540]:
def LSH_Candiadates_List(candidates_list):
    db = pd.DataFrame(candidates_list)#We have 67 Sentences
    db=db[0]
    #print(db)
    #Number of Permutations
    permutations = 2000
    #Number of Recommendations to return
    forest = get_forest(db, permutations)
    num_recommendations = 18
    #query = query_documents[0].lower()
    result = predict(query, db, permutations, num_recommendations, forest)
    #print('\n Top similar sentences \n', result1)


    #Create a list of candiates to be taken to the next layer which is Cosine Simialrity
    candidates=[]
    candidates=result.values
    return candidates

# Print a the candidates list from LSH

In [541]:
LSH_Candidates= LSH_Candiadates_List(raw_documents)

In [543]:
print(len(LSH_Candidates))

17


# 1.2 Cosine Similarity (The second layer)
https://stackoverflow.com/questions/46975929/how-can-i-calculate-the-jaccard-similarity-of-two-lists-containing-strings-in-py


In [544]:
from re import sub
from gensim.utils import simple_preprocess

#query_string = 'fruit and vegetables'
#documents = ['cars drive on the road', 'tomatoes are actually fruit']

stopwords = []

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in LSH_Candidates]
#print(corpus)
query = preprocess(query)
#print (query)

In [545]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

100%|██████████████████████████████████████| 4408/4408 [00:33<00:00, 129.76it/s]


In [546]:
# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def Cosine_Candiadates_List(candidates_list) :
    
    query_tf = tfidf[dictionary.doc2bow(query)]

    index = SoftCosineSimilarity(
                tfidf[[dictionary.doc2bow(document) for document in corpus]],
                similarity_matrix)

    doc_similarity_scores = index[query_tf] 


# Output the sorted similarity scores and documents
    NW_Candiadates=[]
    sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
    for idx in sorted_indexes:
        if doc_similarity_scores[idx]>0.3:
            print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {LSH_Candidates[idx]}')
            NW_Candiadates.append(LSH_Candidates[idx])
    return NW_Candiadates

In [547]:
Pre_NW_Candiadates= Cosine_Candiadates_List(LSH_Candidates)

15 	 0.616 	 Then when it was the ninth night, SHE CONTINUED:

I have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city. It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state. The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up.’ In a low voice, the king replied: ‘Come to me.’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back. With another blow he cut her in two, and threw the two halves on the ground.

When he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking h

In [548]:
NW_Candiadates=[]
NW_Candiadates=Pre_NW_Candiadates[0]#Take the story that has the maximum cosine similarity
print(NW_Candiadates)

Then when it was the ninth night, SHE CONTINUED:

I have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city. It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state. The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up.’ In a low voice, the king replied: ‘Come to me.’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back. With another blow he cut her in two, and threw the two halves on the ground.

When he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking him. The king 

# Send the target Story to be processed again
1. Remove punctuations
3. Lemmatizations(optional)#Removed from the preprocessing operations
4. Tokenization each 3 sentences together


In [549]:
import nltk 
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

def Before_NW_Processing(candidates_list):
    paragraphs = list(filter(lambda x : x != '', candidates_list.split('.')))#https://stackoverflow.com/questions/53240763/python-how-to-separate-paragraphs-from-text
    len(paragraphs)
    paragraphs2=[]
    for i in paragraphs:
        i = i.lower()
        paragraphs2.append(i)
        import re
 
    # Removing punctuations in string
    # Using loop + punctuation string
    final_paragraphs_list=[]
    for ele in paragraphs2:
        x = re.sub(r'[^\w\s]', '', ele)
        x = x.replace('\n'," ")
        final_paragraphs_list.append(x)
   
    #https://stackoverflow.com/questions/50685343/how-to-lemmatize-a-list-of-sentences

    return final_paragraphs_list

In [550]:

Final_Target_Candidates=[]
Final_Target_Candidates=Before_NW_Processing(NW_Candiadates)
#len(Final_Target_Candidates)

query = query_documents[0].lower()
Final_Query_Candidates=[]
Final_Query_Candidates= Before_NW_Processing(query)
#len(Final_Query_Candidates)

In [551]:
len(Final_Query_Candidates)

52

# Tokenization each 3 sentences together


In [552]:
def Three_Sentences_Tokenization(candidates_list):
    Final_Cleaned=candidates_list
    Final_Dataset=[]
    i=0
    if len(Final_Cleaned) % 3==0:
        while(i < len(Final_Cleaned)):
            #if len(x) < len(test):
            if Final_Cleaned[i]==Final_Cleaned[-2]:
                Final_Dataset.append(Final_Cleaned[-2]+" "+Final_Cleaned[-1])
                break
            else:
                Final_Dataset.append(Final_Cleaned[i]+" "+Final_Cleaned[i+1]+" "+Final_Cleaned[i+2])
            i += 3
        return Final_Dataset
    else:
        while(i < len(Final_Cleaned)):
            #if len(x) < len(test):
            if Final_Cleaned[i]==Final_Cleaned[-1]:
                Final_Dataset.append(Final_Cleaned[i])
                Final_Dataset[-1]=Final_Dataset[-2]+" "+Final_Dataset[-1]
                Final_Dataset.remove(Final_Dataset[-2])
                break
            elif Final_Cleaned[i]==Final_Cleaned[-2]:
                Final_Dataset.append(Final_Cleaned[i]+" "+Final_Cleaned[i+1])
                #x.remove(x[-2])
                break
            else:
                Final_Dataset.append(Final_Cleaned[i]+" "+Final_Cleaned[i+1]+" "+Final_Cleaned[i+2])
            i += 3
        return Final_Dataset
        
#print(Final_Query)

In [553]:
Final_Target=[]
Final_Target= Three_Sentences_Tokenization(Final_Target_Candidates)
print(len(Final_Target))
Final_Query=[]
Final_Query= Three_Sentences_Tokenization(Final_Query_Candidates)
print(len(Final_Query))

58
17


# Needlman-Wunch 

In [554]:
from minineedle import needle, core
def needlman_wunch(Target_candidates_list,Query_list,x):
    
    
    R = {} # Dicitonary to Save query, target sentence, precent identity and the actual aligmnet
    IR=x #IndexRange

    query=Query_list[IR].split()
    if IR==0:
        for i in Target_candidates_list[IR:IR+1]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
    else:
        for i in Target_candidates_list[IR-1:IR+1]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
        #print('\n',max_value[3]) 

# Call the Needlman-Wunch with Final_Target and Final Final_Query Datasets

In [555]:
x=0
needlman_wunch(Final_Target,Final_Query,x)

Query:  when it was the ninth night  she said it hath reached me o auspicious king that when the young woman the sorceress took in hand some of the tarn water and spake over it words not to be understood the fishes lifted their heads and stood up on the instant like men the spell on the people of the city having been removed  what was the lake again became a crowded capital the bazars were thronged with folk who bought and sold each citizen was occupied with his own calling and the four hills became islands as they were whilome  then the young woman that wicked sorceress returned to the king and still thinking he was the negro said to him o my love stretch forth thy honoured hand that i may assist thee to rise 

Target Sentence: 
 then when it was the ninth night she continued  i have heard o auspicious king that after the sorceress had taken water from the pool and spoken some unintelligible words over it the fish danced lifted their heads and immediately rose up as the magic spell w

# Now Extract Page Number that has the target Sentece

#  First Let's Read our pdf book Using PymuPDF
https://stackabuse.com/working-with-pdfs-in-python-reading-and-splitting-pages/

In [560]:
#!/usr/bin/python

import fitz

filename = "/Users/ibrahim/Downloads/PenguinFirstVolume.pdf"
search_term = "the sorceress had taken"
pdf_document = fitz.open(filename)

for current_page in range(len(pdf_document)):
    page = pdf_document.loadPage(current_page)
    if page.searchFor(search_term):
        print("found on page",current_page+1)

found on page 107


# First Let's Read our pdf book Using pdfminer
https://pdfminersix.readthedocs.io/en/latest/tutorial/composable.html

In [ ]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open('/Users/ibrahim/Downloads/PenguinFirstVolume.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue())

In [407]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
search_term = "MALCOLM C. LYONS,"

for page_layout in extract_pages("/Users/ibrahim/Downloads/PenguinFirstVolume.pdf"):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            print(element.get_text())

PENGUIN 

 CLASSICS

THE ARABIAN NIGHTS
TALES OF 1001 NIGHTS
VOLUME 1

MALCOLM C. LYONS, sometime Sir Thomas Adams Professor of Arabic at

Cambridge University and a life Fellow of Pembroke College,

Cambridge, is a specialist in the  eld of classical Arabic Literature.

His published works include the biography Saladin: The Politics of the

Holy War, The Arabian Epic: Heroic and Oral Storytelling, Identi cation

and Identity in Classical Arabic Poetry and many articles on Arabic

literature.

URSULA LYONS, formerly an A liated Lecturer at the Faculty of

Oriental Studies at Cambridge University and, since 1976, an

Emeritus Fellow of Lucy Cavendish College, Cambridge, specializes

in modern Arabic literature.

ROBERT IRWIN is the author of For Lust of Knowing: The Orientalists and

Their Enemies, The Middle East in the Middle Ages, The Arabian Nights:

A Companion and numerous other specialized studies of Middle

Eastern politics, art and mysticism. His novels include The Limits of

V

KeyboardInterrupt: 

In [6]:
#!/usr/bin/python

import fitz

filename = "/Users/ibrahim/Downloads/PenguinFirstVolume.pdf"
search_term = "When it was the second night,"
pdf_document = fitz.open(filename)

for current_page in range(len(pdf_document)):
    page = pdf_document.loadPage(current_page)
    if page.searchFor(search_term):
        print("%s found on page %i" % (search_term, current_page+1))

When it was the second night, found on page 51


  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32891 sha256=8ca28e82b51084bb1cafffc4e6de121dc77b1b45394bdc95e8c3c016f2f58396
  Stored in directory: /Users/ibrahim/Library/Caches/pip/wheels/ec/76/38/0e4b92d8a3a89cbfff5be03a40c02d15b2072b1b08ebf28d6a
Successfully built tika
Note: you may need to restart the kernel to use updated packages.


In [7]:
print(page)

[<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(2) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(3) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(4) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(5) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(6) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(7) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(8) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(9) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(10) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(11) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(12) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(13) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(14) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(15) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(16) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(17) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(18) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(19) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(20) 0.000,0.000,612.000,792.000

In [352]:
pip install pdfminer

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install 'pdfminer.six[image]

zsh:1: unmatched '
Note: you may need to restart the kernel to use updated packages.


In [342]:
from pdfminer.high_level import extract_pages
for page_layout in extract_pages("/Users/ibrahim/Downloads/PenguinFirstVolume.pdf"):
    for element in page_layout:
        print(element)

ModuleNotFoundError: No module named 'pdfminer.high_level'

In [341]:
import sys
# if there are no conflicting packages in the default Python Libs =>
sys.path.append("Users/ibrahim/opt/anaconda3/lib/python3.9/site-packages/pdfminer")

In [343]:
pip install pdfminer.high_level

ERROR: Could not find a version that satisfies the requirement pdfminer.high_level (from versions: none)
ERROR: No matching distribution found for pdfminer.high_level
Note: you may need to restart the kernel to use updated packages.


In [124]:
x='Shi‘ite heretic, the man whose corpse is discovered on the gallows\nwould have had the names of the �rst two caliphs Abu Bakr and'
if x in pages[1637]:
    print("Found")
else:
    print("Not Found")

KeyError: 1637

In [82]:
all_pages = []

# work with every page separatelly
for count in range(doc):

    pageObj = readpdf.getPage(count)

    page_text = pageObj.extractText()
    
    page_tokens = word_tokenize(page_text)

    page_keywords = [word for word in page_tokens if not word in stop_words and not word in punctuations]

    page_uppercase_words = [word for word in page_keywords if word.isupper()]

    #all_pages.append( (count, page_keywords, page_uppercase_words) )

    print('page:', count)
    print('keywords:', page_keywords) 
    print('uppercase:', page_uppercase_words)

    # TODO: append/save page to file 

RuntimeError: Directory 'static/' does not exist

In [81]:
pip install frontend

     |████████████████████████████████| 60 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 79 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 58 kB 7.4 MB/s  eta 0:00:01
  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [24]:
print(Final_Query[6])
print("------------------------")
print(Final_Target[8])


 they travelled with fifty mamluks and magnificent gifts and their journey continued day and night for a whole year until as god had decreed their safety they eventually reached their goal  word was sent to the vizier that the king had arrived safe and sound and he together with his soldiers who had despaired of him came to greet him kissing the ground before him and congratulating him on his safe arrival
------------------------
 the king sent for them and married one of the girls himself giving the other to the young man  the fishermans son was made treasurer while the vizier was invested and sent off as ruler of the capital of the black islands the young mans city  with him were sent the fifty mamluks who had come with the king and he was given robes of honour to take to the emirs of the city


In [116]:
from minineedle import needle, core
def needlman_wunch(Target_candidates_list,Query_list,x):
    
    
    R = {} # Dicitonary to Save query, target sentence, precent identity and the actual aligmnet
    IR=x #IndexRange

    query=Query_list[IR]
    if IR==0:
        for i in Target_candidates_list[IR:5]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
    elif IR ==1:
        for i in Target_candidates_list[IR-1:5]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
        #print('\n',max_value[3]) 
    
    elif IR ==2:
        for i in Target_candidates_list[IR-2:5]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
        #print('\n',max_value[3]) 
    elif IR ==3:
        for i in Target_candidates_list[IR-2:IR+5]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
        #print('\n',max_value[3]) 
    else:
        for i in Target_candidates_list[IR-4:IR+5]:
        #print("t--------------",i)
            alignment = needle.NeedlemanWunsch(query, i.split())
            x = alignment.get_identity()
            y = alignment 
            R[i]=[x , query, i, y]
        max_value = max(R.values())
        print('Query: ',Query_list[IR],'\n')
        print('Target Sentence: \n',max_value[2],'\n')
        print('Perecent Identity using NW:',max_value[0])
        #print('\n',max_value[3]) 

In [117]:
x=0
needlman_wunch(Final_Target,Final_Query,x)

KeyboardInterrupt: 

In [113]:
print(Final_Query[1])
print("------------------------")
print(Final_Target[2])


 asked the king what is the fishermans story and she answered by relating the tale of  the fisherman and the jinni  it hath reached me o auspicious king that there was a fisher man well stricken in years who had a wife and three children and withal was of poor condition  now it was his custom to cast his net every day four times and no more
------------------------
 he climbed out delightedly put his clothes back on and went up to the net only to find that what was in it was a dead donkey and that the donkey had made a hole in the net  the fisherman was saddened by this and recited the formula there is no might and no power except with god the exalted the omnipotent before saying this is a strange thing that god has given me by way of food and then reciting  you who court danger diving in the dark of night  give up your efforts do not win your daily bread from god  the fisherman rises to earn his keep  there is the sea with stars woven in the sky   he plunges in buffeted by waves  his 